In [1]:
from multicall import Call
import plotly.express as px
import pandas as pd
from v2_rebalance_dashboard.constants import (
    ROOT_DIR,
    eth_client,
    BALANCER_AURA_DESTINATION_VAULT_ABI,
    balETH_AUTOPOOL_ETH_ADDRESS,
    ROOT_PRICE_ORACLE,
    EXTRA_REWARD_POOL_ABI,
    ERC_20_ABI,
    BASE_REWARD_POOL_ABI,
    AURA_STASH_TOKEN_ABI,
)

from v2_rebalance_dashboard.get_state_by_block import (
    safe_normalize_with_bool_success,
    sync_get_raw_state_by_block_one_block,
    sync_safe_get_raw_state_by_block,
    build_blocks_to_use,
    safe_normalize_6_with_bool_success,
)

# BalancerAuraDestinationVault = "0xa3956D49106288E5c04E6FBbBad5b68593f0bE3b"
# BaseRewardPool4626 = "0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696"  # this is the aura place?
# extra_rewards = "0xe5aABCd2D935C3F06a69b981069A92B252E81a99"  # aura
# stats = "0xE6850bc48feF4F8e4433d95072177aa95cD2fd9F"

vault_df = pd.read_csv("vaults.csv")

AURA = "0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF"
BAL = "0xba100000625a3754423978a60c9317c58a424e3D"


blocks = build_blocks_to_use()


def get_required_addresses_for_balancer_growth_of_a_dollar(row: dict):
    destination_vault_address = row["vaultAddress"]

    try:
        BalancerAuraDestinationVault = eth_client.eth.contract(
            eth_client.toChecksumAddress(destination_vault_address), abi=BALANCER_AURA_DESTINATION_VAULT_ABI
        )
        aura_incentive_stats_address = BalancerAuraDestinationVault.functions.getStats().call()
        auraStaking_address = BalancerAuraDestinationVault.functions.auraStaking().call()

        auraStaking_contract = eth_client.eth.contract(auraStaking_address, abi=BASE_REWARD_POOL_ABI)
        num_extra_rewards = (
            auraStaking_contract.functions.extraRewardsLength().call()
        )  # migh miss things if rewards dip in and out (have not verified)

        extra_rewarder_details = []

        for i in range(num_extra_rewards):
            extra_rewarder_address = auraStaking_contract.functions.extraRewards(i).call()

            extra_reward_token_address = (
                eth_client.eth.contract(extra_rewarder_address, abi=EXTRA_REWARD_POOL_ABI)
                .functions.rewardToken()
                .call()
            )
            extra_reward_token_symbol = (
                eth_client.eth.contract(extra_reward_token_address, abi=ERC_20_ABI).functions.symbol().call()
            )
            extra_reward_token_decimals = (
                eth_client.eth.contract(extra_reward_token_address, abi=ERC_20_ABI).functions.symbol().call()
            )
            if "STASH-" == extra_reward_token_symbol[:6]:
                base_token_address = (
                    eth_client.eth.contract(extra_reward_token_address, abi=AURA_STASH_TOKEN_ABI)
                    .functions.baseToken()
                    .call()
                )
                base_token_symbol = (
                    eth_client.eth.contract(base_token_address, abi=ERC_20_ABI).functions.symbol().call()
                )
            else:
                base_token_address = extra_reward_token_address
                base_token_symbol = extra_reward_token_symbol

            extra_reward_token_decimals = (
                eth_client.eth.contract(base_token_address, abi=ERC_20_ABI).functions.decimals().call()
            )

            extra_rewarder_details.append(
                {
                    "extra_reward_token_address": extra_reward_token_address,
                    "extra_reward_token_symbol": extra_reward_token_symbol,
                    "base_token_address": base_token_address,
                    "base_token_symbol": base_token_symbol,
                    "extra_rewarder_address": extra_rewarder_address,
                    "extra_reward_token_decimals": extra_reward_token_decimals,
                    "auraStaking_address": auraStaking_address,
                }
            )

        return {
            "destination_vault_address": destination_vault_address,
            "aura_incentive_stats_address": aura_incentive_stats_address,
            "auraStaking_address": auraStaking_address,
            "destinationName": row["name"][22:],
            "extra_rewarder_details": extra_rewarder_details,
        }
    except Exception as e:
        return {"destination_vault_address": destination_vault_address, "error": str(e) + str(type(e))}


df = pd.DataFrame.from_records(
    vault_df.apply(get_required_addresses_for_balancer_growth_of_a_dollar, axis=1)
)  # 30 seconds
df

In [ ]:
bal_rewarders_df = df[df["error"].isna()].copy()
extra_rewarder_details = []

for detail_list in bal_rewarders_df["extra_rewarder_details"].values:
    for d in detail_list:
        extra_rewarder_details.append(d)
extra_rewader_df = pd.DataFrame.from_records(extra_rewarder_details)
extra_rewader_df

,extra_reward_token_address,extra_reward_token_symbol,base_token_address,base_token_symbol,extra_rewarder_address,extra_reward_token_decimals,auraStaking_address
0,0x00dfc9ceEFAf596A0Da2e6A1251215a28147EB5b,STASH-LDO,0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32,LDO,0x562B255bE51D0B8e8CA3F16421B2aE7A430Ca358,18,0x59D66C58E83A26d6a0E35114323f65c3945c89c1
1,0x826749CccA0fA780caFB9a1780767e4Cdd712593,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0xc7C1Ad3765a4850C9fcF29b09b30358BF4Ecb315,18,0x59D66C58E83A26d6a0E35114323f65c3945c89c1
2,0xc6065734B898eEdCf450b28Ec2fC5a45a7DCdb2b,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0xf66a72886749c96b18526E8E124cC2e18b7c72D2,18,0xDd1fE5AD401D4777cE89959b7fa587e569Bf125D
3,0xcaB7Ee4EFae2D27add6f5EBB64cdef9d74Beba21,STASH-RPL,0xD33526068D116cE69F19A9ee46F0bd304F21A51f,RPL,0xA4Ebc2d9Cbb8C02eeF5da1dbF2a54B7Ee7576F51,18,0xDd1fE5AD401D4777cE89959b7fa587e569Bf125D
4,0x04e8941897b07B216b3Cc047Df5bEc1EFf48e64C,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0x48CE0d8ce2Af97f9521471caE4eb63141D710B96,18,0xf618102462Ff3cf7edbA4c067316F1C3AbdbA193
5,0x53ecAB28a67A6Cc28A82cE209e1CF4b5d9fd64da,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0x62e6D8dAe7089C8F2f2a5C328c710aa1788742fb,18,0x5F032f15B4e910252EDaDdB899f7201E89C8cD6b
6,0xD3FDfF157A9f2f9E802137f918b2E627f5EA6502,STASH-SWISE,0x48C3399719B582dD63eB5AADf12A40B4C3f52FA2,SWISE,0xC5E75ccd4d40e2Fb280f008f8AFB5EF3415EFA72,18,0x5F032f15B4e910252EDaDdB899f7201E89C8cD6b
7,0xBA45e6500c49570C3C3e3a83C000e47ae1D4C095,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0x8aB6f2574569Bac61B213e7996a88590a1BeAc56,18,0x2a14dB8D09dB0542f6A371c0cB308A768227D67D
8,0x6A881df81d2214D4c62225cd1aA2d86E89B6313f,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0xe5aABCd2D935C3F06a69b981069A92B252E81a99,18,0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696
9,0x87B3E6390C326f3249cd3F5474F2ABBACa5f8691,STASH-AURA,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,AURA,0x656C4A7C3C0F06e3F2aA72fF0e81550713D25EE0,18,0xf8f18dc9E192A9Bf9347DA0E2107d05D5B67F38e


In [ ]:
bal_rewarders_df["extraRewardTokens"] = bal_rewarders_df["extra_rewarder_details"].apply(
    lambda details: [d["base_token_address"] for d in details]
)
bal_rewarders_df["extraRewardersRewardTokenSymbol"] = bal_rewarders_df["extra_rewarder_details"].apply(
    lambda details: [d["base_token_symbol"] for d in details]
)
bal_rewarders_df["extraRewardersRewardTokenDecimals"] = bal_rewarders_df["extra_rewarder_details"].apply(
    lambda details: [d["extra_reward_token_decimals"] for d in details]
)

bal_rewarders_df["extraRewarders"] = bal_rewarders_df["extra_rewarder_details"].apply(
    lambda details: [d["extra_rewarder_address"] for d in details]
)

bal_rewarders_df

,destination_vault_address,error,aura_incentive_stats_address,auraStaking_address,destinationName,extra_rewarder_details,extraRewardTokens,extraRewardersRewardTokenSymbol,extraRewardersRewardTokenDecimals,extraRewarders
5,0x38e73E98d2038FafdC847F13dd9100732383B6F2,NaN,0x93DeeF8ef1922ff3C30abf6EbA85Dc62ADC378db,0x59D66C58E83A26d6a0E35114323f65c3945c89c1,Balancer stETH Stable Pool,[{'extra_reward_token_address': '0x00dfc9ceEFA...,"[0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32, 0...","[LDO, AURA]","[18, 18]","[0x562B255bE51D0B8e8CA3F16421B2aE7A430Ca358, 0..."
6,0xfb1f48a461cCC70081226d8353e45CfBd410dD8F,NaN,0x551050d2dB5043b70598B148e83c9ca16fa21B10,0xDd1fE5AD401D4777cE89959b7fa587e569Bf125D,Balancer rETH Stable Pool,[{'extra_reward_token_address': '0xc6065734B89...,"[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF, 0...","[AURA, RPL]","[18, 18]","[0xf66a72886749c96b18526E8E124cC2e18b7c72D2, 0..."
11,0x37e565f997c2b16d2542E906672E9c6281e77954,NaN,0x085a6456861b7E8B767f6262f0195515e671194F,0xf618102462Ff3cf7edbA4c067316F1C3AbdbA193,Balancer rsETH / ETHx,[{'extra_reward_token_address': '0x04e8941897b...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x48CE0d8ce2Af97f9521471caE4eb63141D710B96]
13,0xA6B62bFdc664Af24DDdfF335A167b867f1d590aF,NaN,0x6733d6Ca76e5238025b743D4922986B90DFb35CA,0x5F032f15B4e910252EDaDdB899f7201E89C8cD6b,Balancer osETH/wETH StablePool,[{'extra_reward_token_address': '0x53ecAB28a67...,"[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF, 0...","[AURA, SWISE]","[18, 18]","[0x62e6D8dAe7089C8F2f2a5C328c710aa1788742fb, 0..."
14,0xF35fbb601e7de870029691f1872D67A8fC866B15,NaN,0x449569489BB8e793EC6838Af97F3ca5C9eC6Bd62,0x2a14dB8D09dB0542f6A371c0cB308A768227D67D,Balancer wstETH-WETH Stable Pool,[{'extra_reward_token_address': '0xBA45e6500c4...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x8aB6f2574569Bac61B213e7996a88590a1BeAc56]
15,0xa3956D49106288E5c04E6FBbBad5b68593f0bE3b,NaN,0xE6850bc48feF4F8e4433d95072177aa95cD2fd9F,0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696,Balancer ETHx/wstETH,[{'extra_reward_token_address': '0x6A881df81d2...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0xe5aABCd2D935C3F06a69b981069A92B252E81a99]
16,0xE9D758002676EecCfAc6106c1E31fE04e419c01D,NaN,0xb8A67FB40AA77466f619fD4b6cf56234Ade4A64b,0xf8f18dc9E192A9Bf9347DA0E2107d05D5B67F38e,Balancer swETH-WETH Stable Pool,[{'extra_reward_token_address': '0x87B3E6390C3...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x656C4A7C3C0F06e3F2aA72fF0e81550713D25EE0]
20,0x0D883F9600857a28CfBccb16b10095EAcF8055af,NaN,0x863254050d3b960C6307dcF7eafdAB2e5e3592C1,0x07A319A023859BbD49CC9C38ee891c3EA9283Cc5,Balancer weETH/rETH StablePool,[{'extra_reward_token_address': '0xDD1D8693e82...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x25d22C5191C67D63AAB70a37FAe06e1c1E1a830F]
22,0x1E02da6E4DFc4875E372104E5e79d54632F52cB3,NaN,0xa975e71B22110cC0FB22554b7cD78bffF4AE16E5,0x95eC73Baa0eCF8159b4EE897D973E41f51978E50,Balancer ezETH-WETH Stable Pool,[{'extra_reward_token_address': '0x72ECf9C9759...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x9674Ecb2BfdB600e264C5010B52974083d35139E]
23,0x8A580F95bF9478C9EC9166E70F5bf2d489C15d8f,NaN,0x27d14558546ffc727320971ccaA48e47c6794d24,0xce98eb8b2Fb98049b3F2dB0A212Ba7ca3Efd63b0,Balancer weETH/ezETH/rswETH,[{'extra_reward_token_address': '0x53dCbeE5Fe3...,[0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF],[AURA],[18],[0x607858e1c3C4a9E462556A7CC752bB2192A42391]


In [ ]:
def _get_safe_price_from_getRangePricesLP(success, value):
    if success:
        spotPriceInQuote, safePriceInQuote, isSpotSafe = value
        return int(safePriceInQuote) / 1e18


def _build_price_calls(rewardTokens: list[str], rewardTokenSymbols: list[str]) -> list[Call]:
    """Returns a list of calls that get the safe ETH value in ETH for each reward token"""
    price_calls = []

    for rewardToken, symbol in zip(rewardTokens, rewardTokenSymbols):
        price_calls.append(
            Call(
                ROOT_PRICE_ORACLE,
                ["getPriceInEth(address)(uint256)", rewardToken],
                [(f"{symbol}_to_ETH", safe_normalize_with_bool_success)],
            )
        )
    return price_calls


def _build_rewardPerToken_calls(
    mainRewarder: str,
    extraRewarders: list[str],
    extraRewardersRewardTokenSymbol: list[str],
    extraRewardersRewardTokenDecimals: list[int],
) -> list[Call]:
    """Get the rewardPerToken for each of the rewarder"""
    bal_rewards_call = Call(
        mainRewarder,
        ["rewardPerToken()(uint256)"],
        [("mintedBAL_rewardPerToken", safe_normalize_with_bool_success)],
    )

    aura_minted_from_one_BAL_call = Call(
        "0x551050d2dB5043b70598B148e83c9ca16fa21B10",  # just has to be an incentive stats contract for AURA-BAL this is an old one
        [
            "getPlatformTokenMintAmount(address,uint256)(uint256)",
            AURA,
            int(1e18),
        ],  # imo rounding error, 1.0936 to 1.0696 in a 50 days 1:1 would be 95% as close
        [("AURA_minted_for_one_BAL", safe_normalize_with_bool_success)],
    )

    rewardPerTokenCalls = []
    for i, (extraRewarder, extraRewardTokenSymbol, decimals) in enumerate(
        zip(extraRewarders, extraRewardersRewardTokenSymbol, extraRewardersRewardTokenDecimals)
    ):
        if decimals == 18:
            func = safe_normalize_with_bool_success
        elif decimals == 6:
            func = safe_normalize_with_bool_success
        rewardPerTokenCall = Call(
            extraRewarder,
            ["rewardPerToken()(uint256)"],
            [
                (f"{extraRewardTokenSymbol}|| {i} extraRewardPerToken", safe_normalize_with_bool_success)
            ],  # using `i` here to avoid duplicates
        )
        rewardPerTokenCalls.append(rewardPerTokenCall)

    return [bal_rewards_call, aura_minted_from_one_BAL_call, *rewardPerTokenCalls]


def build_growth_of_a_dollar_calls(
    rewardTokens: list[str],
    rewardTokenSymbols: list[str],
    mainRewarder: str,
    extraRewarders: list[str],
    extraRewardersRewardTokenSymbols: list[str],
    extraRewardersRewardTokenDecimals: list[int],
    balancerAuraDestinationVault_address: str,
) -> list[Call]:
    price_calls = _build_price_calls(rewardTokens, rewardTokenSymbols)
    reward_per_token_calls = _build_rewardPerToken_calls(
        mainRewarder,
        extraRewarders,
        extraRewardersRewardTokenSymbols,
        extraRewardersRewardTokenDecimals,
    )

    get_safe_lp_token_value_call = Call(
        balancerAuraDestinationVault_address,
        ["getRangePricesLP()((uint256,uint256,bool))"],
        [("safeLPTokenPriceInETH", _get_safe_price_from_getRangePricesLP)],
    )

    return [*price_calls, *reward_per_token_calls, get_safe_lp_token_value_call]


def transform_growth_of_a_dollar_df_to_growth_of_1_ETH_with_no_entrance_costs(
    raw_growth_of_a_dollar_df: pd.DataFrame, mock_date_deployed: str = "2024, 7, 17"
):
    growth_of_a_dollar_df = raw_growth_of_a_dollar_df[raw_growth_of_a_dollar_df.index > mock_date_deployed].copy()
    growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] = (
        1 / growth_of_a_dollar_df["safeLPTokenPriceInETH"].values[0]
    )  # constant
    growth_of_a_dollar_df["safe_value_of_lp_tokens"] = (
        growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * growth_of_a_dollar_df["safeLPTokenPriceInETH"]
    )

    _add_BAL_and_AURA_minted_eth_value(growth_of_a_dollar_df)
    _add_extra_rewarders_incentive_eth_value(growth_of_a_dollar_df)
    growth_of_a_dollar_df["total_cumulative_incentive_tokens_current_eth_value"] = (
        growth_of_a_dollar_df["extra_rewarder_cumulative_incentive_eth_value"]
        + growth_of_a_dollar_df["main_rewarder_cumulative_incentive_eth_value"]
    )
    growth_of_a_dollar_df["growth_of_1_ETH"] = (
        growth_of_a_dollar_df["safe_value_of_lp_tokens"]
        + growth_of_a_dollar_df["total_cumulative_incentive_tokens_current_eth_value"]
    )
    return growth_of_a_dollar_df


def _add_BAL_and_AURA_minted_eth_value(growth_of_a_dollar_df: pd.DataFrame):
    starting_mintedBAL_rewardPerToken = growth_of_a_dollar_df["mintedBAL_rewardPerToken"].values[0]
    growth_of_a_dollar_df["BAL_minted_since_start_date"] = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
        growth_of_a_dollar_df["mintedBAL_rewardPerToken"] - starting_mintedBAL_rewardPerToken
    )
    growth_of_a_dollar_df["AURA_minted_since_start_date"] = (
        growth_of_a_dollar_df["BAL_minted_since_start_date"] * growth_of_a_dollar_df["AURA_minted_for_one_BAL"]
    )
    growth_of_a_dollar_df["main_rewarder_cumulative_incentive_eth_value"] = (
        growth_of_a_dollar_df["BAL_minted_since_start_date"] * growth_of_a_dollar_df["BAL_to_ETH"]
    ) + (growth_of_a_dollar_df["AURA_minted_since_start_date"] * growth_of_a_dollar_df["AURA_to_ETH"])


def _add_extra_rewarders_incentive_eth_value(growth_of_a_dollar_df: pd.DataFrame):
    extra_reward_per_token_cols = [
        c for c in growth_of_a_dollar_df.columns if "extraRewardPerToken" in c
    ]  # format like AURA|| 0 extraRewardPerToken
    extra_reward_to_eth_cols = [f"{c.split('||')[0]}_to_ETH" for c in extra_reward_per_token_cols]
    print(f"{extra_reward_per_token_cols=} {extra_reward_to_eth_cols=}")
    growth_of_a_dollar_df["extra_rewarder_cumulative_incentive_eth_value"] = 0.0
    for extra_rewardPerToken_col, ETH_price_col in zip(extra_reward_per_token_cols, extra_reward_to_eth_cols):
        starting_rewardPerToken = growth_of_a_dollar_df[extra_rewardPerToken_col].values[0]
        extra_rewards_minted_since_start = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
            growth_of_a_dollar_df[extra_rewardPerToken_col] - starting_rewardPerToken
        )
        growth_of_a_dollar_df["extra_rewarder_cumulative_incentive_eth_value"] += (
            extra_rewards_minted_since_start * growth_of_a_dollar_df[ETH_price_col]
        )


dfs = []
for (
    destinationName,
    destination_vault_address,
    auraStaking_address,
    extraRewarders,
    extraRewardTokens,
    extraRewardersRewardTokenSymbol,
    extraRewardersRewardTokenDecimals,
) in zip(
    bal_rewarders_df["destinationName"],
    bal_rewarders_df["destination_vault_address"],
    bal_rewarders_df["auraStaking_address"],
    bal_rewarders_df["extraRewarders"],
    bal_rewarders_df["extraRewardTokens"],
    bal_rewarders_df["extraRewardersRewardTokenSymbol"],
    bal_rewarders_df["extraRewardersRewardTokenDecimals"],
):
    mainRewarder = auraStaking_address
    rewardTokens = [BAL, AURA, *extraRewardTokens]
    rewardTokenSymbols = ["BAL", "AURA", *extraRewardersRewardTokenSymbol]

    growth_of_a_dollar_calls = build_growth_of_a_dollar_calls(
        rewardTokens=rewardTokens,
        rewardTokenSymbols=rewardTokenSymbols,
        mainRewarder=mainRewarder,
        extraRewarders=extraRewarders,
        extraRewardersRewardTokenSymbols=extraRewardersRewardTokenSymbol,
        extraRewardersRewardTokenDecimals=extraRewardersRewardTokenDecimals,
        balancerAuraDestinationVault_address=destination_vault_address,
    )
    raw_growth_of_a_dollar_df = sync_safe_get_raw_state_by_block(growth_of_a_dollar_calls, blocks)
    raw_growth_of_a_dollar_df = raw_growth_of_a_dollar_df[raw_growth_of_a_dollar_df.index > "2024, 7, 17"].copy()
    growth_of_a_dollar_df = transform_growth_of_a_dollar_df_to_growth_of_1_ETH_with_no_entrance_costs(
        raw_growth_of_a_dollar_df
    )
    growth_of_a_dollar_df[f"{destinationName} Growth of a ETH"] = growth_of_a_dollar_df['growth_of_1_ETH']
    fig = px.line(growth_of_a_dollar_df[f"{destinationName} Growth of a ETH"])
    fig.show()
    dfs.append(growth_of_a_dollar_df)

extra_reward_per_token_cols=['LDO|| 0 extraRewardPerToken', 'AURA|| 1 extraRewardPerToken'] extra_reward_to_eth_cols=['LDO_to_ETH', 'AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken', 'RPL|| 1 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH', 'RPL_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken', 'SWISE|| 1 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH', 'SWISE_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


...

extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken', 'USDC|| 1 extraRewardPerToken', 'USDT|| 2 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH', 'USDC_to_ETH', 'USDT_to_ETH']


# can we price both usd and usdt with the root price oracle

0xdAC17F958D2ee523a2206206994597C13D831ec7, the root price oracle can't price usdt as of Sep 3, 2024

In [ ]:
nav_per_share_call = Call(
    balETH_AUTOPOOL_ETH_ADDRESS,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [('balETH', safe_normalize_with_bool_success)],
)
nav_per_share_df = sync_safe_get_raw_state_by_block([nav_per_share_call], blocks) 
nav_per_share_df

In [ ]:
growth_df = pd.concat([d[d.columns[-1]].astype(float) for d in dfs], axis=1)
growth_df['balETH'] = nav_per_share_df['balETH']
px.line(growth_df)

NameError: name 'balETH' is not defined

In [ ]:
#


raw_growth_of_a_dollar_df
# growth_of_a_dollar_df= growth_of_a_dollar_df[growth_of_a_dollar_df.index > '2024, 7, 17'].copy()

In [ ]:
growth_of_a_dollar_df = transform_groth_of_a_dollar_df_to_growth_of_1_ETH_with_no_entrance_costs(
    raw_growth_of_a_dollar_df
)
px.line(growth_of_a_dollar_df["growth_of_1_ETH"])

In [ ]:
growth_of_a_dollar_df

In [ ]:
break

In [ ]:
growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] = 1 / growth_of_a_dollar_df["safeLPTokenPriceInETH"].values[0]
growth_of_a_dollar_df["safe_value_of_lp_tokens"] = (
    growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * growth_of_a_dollar_df["safeLPTokenPriceInETH"]
)

# need to add some assumptions in here about the gas costs of entering, (that we paid) ie 200 usd
# and the slippage cost to enter? using our values lets say
starting_bal_reward_per_token = growth_of_a_dollar_df["BAL_reward_per_token"].values[0]
starting_stash_aura_reward_per_token = growth_of_a_dollar_df["stash_aura_reward_per_token"].values[0]

growth_of_a_dollar_df["bal_earned_since_deposit"] = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
    growth_of_a_dollar_df["BAL_reward_per_token"] - starting_bal_reward_per_token
)
growth_of_a_dollar_df["aura_minted_since_deposit"] = (
    growth_of_a_dollar_df["bal_earned_since_deposit"] * growth_of_a_dollar_df["AURA_minted_for_one_BAL"]
)

growth_of_a_dollar_df["stash_aura_earned_since_deposit"] = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
    growth_of_a_dollar_df["stash_aura_reward_per_token"] - starting_stash_aura_reward_per_token
)

growth_of_a_dollar_df["current_eth_value_of_incentive_tokens_earned"] = (
    (growth_of_a_dollar_df["bal_earned_since_deposit"] * growth_of_a_dollar_df["BAL_to_ETH"])
    + (growth_of_a_dollar_df["aura_minted_since_deposit"] * growth_of_a_dollar_df["AURA_to_ETH"])
    + (growth_of_a_dollar_df["stash_aura_earned_since_deposit"] * growth_of_a_dollar_df["AURA_to_ETH"])
)
growth_of_a_dollar_df["wstETH_ETHx_balancer_value_of_one_eth"] = (
    growth_of_a_dollar_df["safe_value_of_lp_tokens"]
    + growth_of_a_dollar_df["current_eth_value_of_incentive_tokens_earned"]
)
px.line(
    growth_of_a_dollar_df[["wstETH_ETHx_balancer_value_of_one_eth", "our_nav_per_share"]],
    title="Nav Per Share vs Balancer wstETH/ETHx",
)

In [ ]:
# account for costs

growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] = (
    1 / growth_of_a_dollar_df["safeLPTokenPriceInETH"].values[0]
) * (
    1 - 0.0035
)  # assume 35 bips to enter and 10bips of gas cost
growth_of_a_dollar_df["safe_value_of_lp_tokens"] = (
    growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * growth_of_a_dollar_df["safeLPTokenPriceInETH"]
)

# need to add some assumptions in here about the gas costs of entering, (that we paid) ie 200 usd
# and the slippage cost to enter? using our values lets say
starting_bal_reward_per_token = growth_of_a_dollar_df["BAL_reward_per_token"].values[0]
starting_stash_aura_reward_per_token = growth_of_a_dollar_df["stash_aura_reward_per_token"].values[0]

growth_of_a_dollar_df["bal_earned_since_deposit"] = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
    growth_of_a_dollar_df["BAL_reward_per_token"] - starting_bal_reward_per_token
)
growth_of_a_dollar_df["aura_minted_since_deposit"] = (
    growth_of_a_dollar_df["bal_earned_since_deposit"] * growth_of_a_dollar_df["AURA_minted_for_one_BAL"]
)

growth_of_a_dollar_df["stash_aura_earned_since_deposit"] = growth_of_a_dollar_df["starting_quantity_of_lp_tokens"] * (
    growth_of_a_dollar_df["stash_aura_reward_per_token"] - starting_stash_aura_reward_per_token
)

growth_of_a_dollar_df["current_eth_value_of_incentive_tokens_earned"] = (
    (growth_of_a_dollar_df["bal_earned_since_deposit"] * growth_of_a_dollar_df["BAL_to_ETH"])
    + (growth_of_a_dollar_df["aura_minted_since_deposit"] * growth_of_a_dollar_df["AURA_to_ETH"])
    + (growth_of_a_dollar_df["stash_aura_earned_since_deposit"] * growth_of_a_dollar_df["AURA_to_ETH"])
)

growth_of_a_dollar_df["current_eth_value_of_incentive_tokens_earned"] = growth_of_a_dollar_df[
    "current_eth_value_of_incentive_tokens_earned"
]  # assume 35 ps of cost to sell reward tokens?
growth_of_a_dollar_df["wstETH_ETHx_balancer_value_of_one_eth"] = (
    growth_of_a_dollar_df["safe_value_of_lp_tokens"]
    + growth_of_a_dollar_df["current_eth_value_of_incentive_tokens_earned"]
)
px.line(
    growth_of_a_dollar_df[["wstETH_ETHx_balancer_value_of_one_eth", "our_nav_per_share"]],
    title="Nav Per Share vs Balancer wstETH/ETHx if it cost 35bps to enter",
)

In [ ]:
(20.40 / 20.42) - 1  # 10 bips
(59.79 / 60) - 1  # 35 bips +
((35 * 60) + (10 * 20)) / 80
# avg of 29 bips of slippage give them

In [ ]:
# # BaseRewardPool4626 = '0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696' # this is the aura place?
# # extra_rewards = '0xe5aABCd2D935C3F06a69b981069A92B252E81a99' # aura
# # assumes no dilution, at 1 ETH level.
# # Our records are real so we dilute the returns, this is hypothetical so without adding a bunch of complexity


# def _get_safe_price_from_getRangePricesLP(success, value):
#     if success:
#         spotPriceInQuote, safePriceInQuote, isSpotSafe = value
#         return int(safePriceInQuote) / 1e18


# def build_reward_per_token_stored_calls():
#     AURA = "0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF"
#     BAL = "0xba100000625a3754423978a60c9317c58a424e3D"
#     bal_rewards_call = Call(
#         BaseRewardPool4626,
#         ["rewardPerToken()(uint256)"],
#         [("BAL_reward_per_token", safe_normalize_with_bool_success)],
#     )

#     aura_stash_rewards_call = Call(
#         extra_rewards,
#         ["rewardPerToken()(uint256)"],
#         [("stash_aura_reward_per_token", safe_normalize_with_bool_success)],
#     )

#     aura_minted_from_one_BAL_call = Call(
#         stats,
#         [
#             "getPlatformTokenMintAmount(address,uint256)(uint256)",
#             AURA,
#             int(1e18),
#         ],  # imo rounding error, 1.0936 to 1.0696 in a 50 days 1:1 would be 95% as close
#         [("AURA_minted_for_one_BAL", safe_normalize_with_bool_success)],
#     )

#     get_safe_lp_token_value_call = Call(
#         BalancerAuraDestinationVault,
#         ["getRangePricesLP()((uint256,uint256,bool))"],
#         [("safeLPTokenPriceInETH", _get_safe_price_from_getRangePricesLP)],
#     )
#     price_calls = []
#     for rewardToken, name in zip([AURA, BAL], ["AURA_to_ETH", "BAL_to_ETH"]):
#         price_calls.append(
#             Call(
#                 ROOT_PRICE_ORACLE,
#                 ["getPriceInEth(address)(uint256)", rewardToken],
#                 [(name, safe_normalize_with_bool_success)],
#             )
#         )

#     our_nav_per_share = Call(
#         balETH_AUTOPOOL_ETH_ADDRESS,
#         ["convertToAssets(uint256)(uint256)", int(1e18)],
#         [('our_nav_per_share', safe_normalize_with_bool_success)],
#     )

#     return [bal_rewards_call, aura_minted_from_one_BAL_call, aura_stash_rewards_call, get_safe_lp_token_value_call, *price_calls, our_nav_per_share]


# calls = build_reward_per_token_stored_calls()
# blocks = build_blocks_to_use()
# response = sync_get_raw_state_by_block_one_block(calls, 20642289)
# growth_of_a_dollar_df = sync_safe_get_raw_state_by_block(calls, blocks)
# growth_of_a_dollar_df= growth_of_a_dollar_df[growth_of_a_dollar_df.index > '2024, 7, 17'].copy()